$$\renewcommand{\ket}[1]{\left|{#1}\right\rangle}$$

<div align=center>

### Open me with [Google Colab](https://colab.research.google.com/) for a better experience!
Also, collapse all the code cells to have a clear view of the notebook.

</div>

$$\renewcommand{\bra}[1]{\left\langle{#1}\right|}$$

---
# Initialization

In [653]:
# Check, install and import requirements and define utility functions.
#@title ###### Check, install and import requirements and define utility functions.

import pkg_resources
from pkg_resources import DistributionNotFound, VersionConflict
import os

dependencies = [
  'numpy',
  'matplotlib'
]

try:
    pkg_resources.require(dependencies)
except DistributionNotFound:
    os.system("pip install numpy")
    os.system("pip install matplotlib")

import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg

from copy import deepcopy
from IPython.display import display as disp, Math, Latex
from ipywidgets import widgets, interact, interactive

def as_latex_matrix(m, replace_zeros = None):
  return r"\begin{pmatrix}"+" \\\\ ".join([" & ".join([(str(i) if i != 0 or not replace_zeros else str(replace_zeros)) for i in j]) for j in m])+"\end{pmatrix}"

def as_latex_vector(v, replace_zeros = None):
  return r"\begin{pmatrix}"+" \\\\ ".join([f"{i:.3f}" if np.round(i,3) != 0 or not replace_zeros else str(replace_zeros)for i in v])+"\end{pmatrix}"


def print_latex(code):
  disp(Math(code))

In [654]:
# Include base code (given in subject)
#@title ###### Include base code (given in subject)

import numpy as np
import scipy as sp
from numpy import linalg as la
import matplotlib.pyplot as plt
from numpy import linalg as LA;
from scipy import linalg as LA2;
from numpy import random as rand
from scipy.sparse import diags

def tensorvect(a,b):
    return(np.tensordot(a,b,axes=0).flatten())

def tensorvectop(a,b):
    return LA2.kron(a,b)

def opchain(a,i,nspin):
    if i==1:
        return LA2.kron(a,np.identity(2**(nspin-1)))
    else:
        if i==nspin:
            return LA2.kron(np.identity(2**(nspin-1)),a)
        else:
            return LA2.kron(LA2.kron(np.identity(2**(i-1)),a),np.identity(2**(nspin-i)))
        
def opchain2(a,i,b,j,nspin):
    if i==1:
        if j==nspin:
            return LA2.kron(LA2.kron(a,np.identity(2**(nspin-2))),b)
        else:
            return LA2.kron(LA2.kron(a,np.identity(2**(j-2))),LA2.kron(b,np.identity(2**(nspin-j))))      
    else:
        if j==nspin:
            return LA2.kron(LA2.kron(np.identity(2**(i-1)),a),LA2.kron(np.identity(2**(nspin-(i+1))),b))
        else:
            return LA2.kron(LA2.kron(LA2.kron(np.identity(2**(i-1)),a),LA2.kron(np.identity(2**(j-(i+1))),b)),np.identity(2**(nspin-j)))
            
def buildstate(bin):
    v=[0. for i in range(2**len(bin))];
    v[int(bin,2)]=1.
    return np.array(v)


def diracrep(psi,nspin):
    state='';
    for i in range(2**nspin):
        if abs(psi[i])>10**(-6):
            state=state+'+'+str(psi[i])+'|'+format(i,'0'+str(nspin)+'b')+'>'
    return state

def binnum(n):
    l=['0','1'];
    if n==1:
        return l
    else:
        return ['0'+i for i in binnum(n-1)]+['1'+i for i in binnum(n-1)]
    
def densmat(psi,i,nspin):
    if i>1:
        listindex0=binnum(i-1)
        listindex0=[j+'0' for j in listindex0]
    else:
        listindex0=['0']
    if i<nspin:
        listcomp=binnum(nspin-i)
        listindex0=list(np.array([[j+k for k in listcomp] for j in listindex0]).flatten())
    if i>1:
        listindex1=binnum(i-1)
        listindex1=[j+'1' for j in listindex1]
    else:
        listindex1=['1']
    if i<nspin:
        listcomp=binnum(nspin-i)
        listindex1=list(np.array([[j+k for k in listcomp] for j in listindex1]).flatten())
    rho00=sum(psi[int(j,2)]*np.conjugate(psi[int(j,2)]) for j in listindex0)
    rho11=sum(psi[int(j,2)]*np.conjugate(psi[int(j,2)]) for j in listindex1)
    rho01=sum(psi[int(j,2)]*np.conjugate(psi[int(listindex1[listindex0.index(j)],2)]) for j in listindex0)
    return np.array([[rho00,rho01],[np.conjugate(rho01),rho11]])

def avdensmat(psi,nspin):
    rho=densmat(psi,1,nspin);
    if nspin>1:
        for i in range(2,nspin+1):
            rho=rho+densmat(psi,i,nspin)
    rho=rho/nspin;
    return rho

def purity(rho):
    rho2 = np.dot(rho,rho)
    tr = np.trace(rho2)
    return(tr)

def SvN(rho):
    vp=np.real(LA.eigvals(rho));
    S=0.;
    for i in range(len(vp)):
        if vp[i]>0.:
            S=S+vp[i]*np.log(vp[i])
    return -S

def entangl(psi,nspin):
    S=SvN(densmat(psi,1,nspin));
    if nspin>1:
        for i in range(2,nspin+1):
            S=S+SvN(densmat(psi,i,nspin))
    return S/nspin

def Disorder(psi,nspin):
    return SvN(avdensmat(psi,nspin))-entangl(psi,nspin)

sigX=np.array([[0.,1.],[1.,0.]]);
sigY=np.array([[0.,-1j],[1j,0.]]);
sigZ=np.array([[1.,0.],[0.,-1.]]);
sig1=np.array([[1.,0.],[0.,0.]]);
id2 =np.array([[1.,0.],[0.,1.]]);

NOT=sigX;
HAD1=np.array([[1./np.sqrt(2.),1./np.sqrt(2.)],[1./np.sqrt(2.),-1./np.sqrt(2.)]]);
CNOT=np.array([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,0.,1.],[0.,0.,1.,0.]]);
HAD2=np.array([[0.5,0.5,0.5,0.5],[0.5,-0.5,0.5,-0.5],[0.5,0.5,-0.5,-0.5],[0.5,-0.5,-0.5,0.5]]);
SWAP=np.array([[1.,0.,0.,0.],[0.,0.,1.,0.],[0.,1.,0.,0.],[0.,0.,0.,1.]]);

---
# Simulation parameters

In [655]:
# Definition of simulation parameters
#@title ###### Definition of simulation parameters

J_value = 1 #@param {type:"number"}

Jx_value = 1 #@param {type:"number"}
Jy_value = 1 #@param {type:"number"}
Jz_value = 1 #@param {type:"number"}

N = 3 #@param {type:"number"}

---
# Chapter 1

### Constants
We set
$$
\hbar = 1\\
\omega = 0.5
$$

In [656]:
# Defining constants
#@title ###### Defining constants

w = 0.5 #@param {type:"number"}
hbar = 1 #@param {type:"number"}

### Pauli matrices
We know the Pauli matrices:

$$
\sigma_x = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}
$$

$$
\sigma_y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}
$$

$$
\sigma_z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}
$$

In [657]:
# Definition of pauli matrices
#@title ###### Definition of pauli matrices

sigma = np.array([
    [[0, 1],
     [1, 0]],

    [[0, -1j],
     [1j,  0]],
    
    [[1,  0],
     [0, -1]]

])

### Spins
We can deduce the spin matrices

$$
S_x = \frac \hbar 2 \sigma_x = \ket 0 \bra 1 + \ket 1 \bra 0    
$$

$$
S_y = \frac \hbar 2 \sigma_y = \ket 0 \bra 1 - \ket 1 \bra 0
$$

$$
S_z = \frac \hbar 2 \sigma_z = \ket 0 \bra 0 - \ket 1 \bra 1
$$

In [658]:
# Definition of the spin matrices
#@title ###### Definition of the spin matrices

S = [hbar/2 * sigma[i] for i in range(3)]

#print(S[0], "\n\n", S[1], "\n\n", S[2])

### $h_0$
And then, $h_0$ is defined such as:
$$
h_0 = - \gamma B S_z =
\begin{pmatrix}
\frac \omega 2 & ... & 0 \\
... & \searrow & ... \\
0 & ... & \frac \omega 2
\end{pmatrix}
$$

$$
\rightarrow h_0 = \begin{pmatrix} -\omega & 0 \\ 0 & 0 \end{pmatrix} D
$$

In [659]:
# Definition of h_0 and I
#@title ###### Definition of $h_0$ and $I$

h_0 = np.array([
    [-w, 0],
    [ 0, 0]
])

I = np.eye(N)

### Hamiltonian
If we consider a system with several spins, the total hamiltonian is the sum of the hamiltonians of each spin and the interaction between them.

For exemple, in the case of 2 spins:

$$
H_T = H_1 + H_2 + H_{12} = H_0 + H_{int}
$$

Where

$$
H_1 = h_0 \otimes I_z = \begin{pmatrix} -\omega & 0 \\ 0 & 0 \end{pmatrix} \otimes \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} = \begin{pmatrix} -\omega & 0 & 0 & 0 \\ 0 & -\omega & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \end{pmatrix}
$$

$$
H_2 = I_2 \otimes h_0 = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \otimes \begin{pmatrix} -\omega & 0 \\ 0 & 0 \end{pmatrix} = \begin{pmatrix} -\omega & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & -\omega & 0 \\ 0 & 0 & 0 & 0 \end{pmatrix}
$$

And $H_{12}$ is defined such as:
$$
\begin{array}{rl}
H_{int} &= - \sum_{i=1}^N \sum_{j>i} \vec J_{ij} \vec S_i \odot \vec S_j\\
&= - \sum_{i=1}^N \sum_{j>i} \sum_{u \in {x,y,z}} \vec J_{ij}^u \vec S_i^u\vec S_j^u\\
\end{array}
$$

In [660]:
# Definition of H_int
#@title ###### Definition of H_int

def get_H_int(J, S, verbose = False):

    # Getting the number of spins
    _, N, _ = J.shape

    # Initalizing the Hamiltonian with a matrixc of size*size filled of 0
    H = np.zeros((2**N, 2**N), dtype=complex)

    # Sums
    for i in range(1,N): # 1 to N-1
        for j in range(i+1, N+1): # i+1 to N
            for u in range(3): # 0 to 2
                H -= J[u,i-1,j-1] * opchain2(S[u], i, S[u], j, N) # cf. eq 1.6

                # Debug
                if verbose:
                    print("\n------------------------------\n")
                    print("i j u J[u,i,j]")
                    print(i,j, ["x","y","z"][u], J[u,i,j])
                    print(" ")
                    print(J[u,i,j] * opchain2(S[u], i+1, S[u], j+1, N).astype(float))
    
    return H

---
# Chapter 2

## 2.1 - Studied models

> **ToDo:** Build with python the quantum Hamiltonian $H$ as a $2^N × 2^N$ array for each model. In the final report, give all the Hamiltonians for the cases N = 3 and N = 8


### Models

<div align=center>

#### Ferromagnetic

$$
\forall i,j \in \{1,...,N\} \:\:\:\:\: J_{ij, closed}^u \begin{cases}
    \geq 0 & \text{if } j = i+1 \quad \forall u\\
    = 0 & \text{otherwise}
\end{cases}
$$

####Antiferromagnetic

$$
\forall i,j \in \{1,...,N\} \:\:\:\:\: J_{ij, closed}^u \begin{cases}
    \leq 0 & \text{if } j = i+1 \quad \forall u\\
    = 0 & \text{otherwise}
\end{cases}
$$

#### Open Ising model

$$
\forall i,j \in \{1,...,N\}\:\:\:\:\:  J_{ij, open}^u = \begin{cases}
    J & \text{if } u = z \text{ and } j = i+1 \:\:\:\:\: \text{ (each atom interact with the next one)}\\
    0 & \text{if } u = \{x,y\} \end{cases}
$$


#### Closed Ising model

$$
\forall i,j \in \{1,...,N\} \:\:\:\:\: J_{ij, closed}^u = \begin{cases}
    J & \text{if } u = z \text{ and } j = i+1 \:\:\:\:\: &\text{ (each atom interact with the next one)}\\
    J & \text{if } j = N \text{ and } i=1 \:\:\:\:\: &\text{ (last atom interact with the first one)}\\
    0 & \text{otherwise}\end{cases}  
$$


#### Heisenberg XXX

$$
\forall i,j \in \{1,...,N\} \:\:\:\:\: J_{ij, closed}^u = \begin{cases}
    J_x & \text{if } j = i+1\\
    0 & \text{otherwise}
\end{cases}
$$

#### Heisenberg XXZ

$$
\forall i,j \in \{1,...,N\} \:\:\:\:\: J_{ij, closed}^u = \begin{cases}
    J_x & \text{if } u = {x,y} \text{ and } j = i+1\\
    J_z & \text{if } u = z \text{ and } j = i+1\\
    0 & \text{otherwise}
\end{cases}
$$

#### Heisenberg XYZ

$$
\forall i,j \in \{1,...,N\} \:\:\:\:\: J_{ij, closed}^u = \begin{cases}
    J_x & \text{if } u = x \text{ and } j = i+1\\
    J_y & \text{if } u = y \text{ and } j = i+1\\
    J_z & \text{if } u = z \text{ and } j = i+1\\
    0 & \text{otherwise}
\end{cases}
$$

</div>

In [661]:
#Computing models
#@title ###### Computing J using opened Ising models

i,j = np.meshgrid(np.arange(N), np.arange(N))

# Ferromatic and antiferromatic

Jferro = (np.random.rand(3*N*N).reshape((3,N,N)) + 1) * (j+1 == i)
Jantiferro = - Jferro

# Open Ising model

Jx = np.zeros((3,N,N))
Jy = np.zeros((3,N,N))
Jz = np.zeros((3,N,N))

Jx[0,:,:] = J_value * (j+1 == i) # Ising X model -> J = J on direction x and when j = i+1 
Jy[1,:,:] = J_value * (j+1 == i) # Ising Z model -> J = J on direction y and when j = i+1 
Jz[2,:,:] = J_value * (j+1 == i) # Ising Z model -> J = J on direction z and when j = i+1 

# Closed Ising model

JxC = np.copy(Jx)
JzC = np.copy(Jz)
JyC = np.copy(Jy)

JxC[0,0,-1] = J_value
JyC[1,0,-1] = J_value
JzC[2,0,-1] = J_value

# Heisenberg XXX
Jhxxx = np.ones((3,N,N)) * (j+1 == i)

# Heisenberg XXZ
Jhxxz = np.ones((3,N,N)) * (j+1 == i)
Jhxxz[:1,:,:] *= Jx_value
Jhxxz[2,:,:] *= Jz_value

# Heisenberg XYZ
Jhxyz = np.ones((3,N,N)) * (j+1 == i)
Jhxyz[0,:,:] *= Jx_value
Jhxyz[1,:,:] *= Jy_value
Jhxyz[2,:,:] *= Jz_value


display = "Heisenberg XXX" #@param ["None", "Insing-X opened", "Insing-Y opened", "Insing-Z opened", "Insing-X closed", "Insing-Y closed", "Insing-Z closed", "Heisenberg XXX", "Heisenberg XXZ", "Heisenberg XYZ"] {type:"string"}

selection = {
    "Insing-X opened":Jx,
    "Insing-Y opened":Jy,
    "Insing-Z opened":Jz,
    "Insing-X closed":JxC,
    "Insing-Y closed":JyC,
    "Insing-Z closed":JzC,
    "Heisenberg XXX":Jhxxx,
    "Heisenberg XXZ":Jhxxz,
    "Heisenberg XYZ":Jhxyz
}

if display == "None":
    J = JzC
else:
    J = selection[display]

    print(f"For {display} model:")
    print_latex(r"J_x = " + as_latex_matrix(J[0].astype(int)) + r"\:\:\:\:\:"
              + r"J_y = " + as_latex_matrix(J[1].astype(int)) + r"\:\:\:\:\:"
              + r"J_z = " + as_latex_matrix(J[2].astype(int))
    )



For Heisenberg XXX model:


<IPython.core.display.Math object>

### Free Hamiltonian
We remember that the free hamiltonians for the i-st spin is:

$$
H_i = I_1 \otimes ... \otimes I_{i-1} \otimes h_0 \otimes I_{i+1} \otimes... \otimes I_N
$$
$$\rightarrow h_0 \text{ is at position } i$$

In [662]:
# Computing free Hamiltonian
#@title ###### Computing free Hamiltonian

H0_LIST = [opchain(h_0, i+1, N) for i in range(N)]
H0 = np.sum(H0_LIST, axis=0)

if display != "None":
  print_latex(r"H_0 = " + as_latex_matrix(np.real(H0), replace_zeros="0"))

<IPython.core.display.Math object>

### Interaction Hamiltonian
We recall that the interaction Hamiltonian is defined such as:

$$
\begin{array}{rl}
H_{int} &= - \sum_{i=1}^N \sum_{j>i} \vec J_{ij} \vec S_i \odot \vec S_j\\
&= - \sum_{i=1}^N \sum_{j>i} \sum_{u \in {x,y,z}} \vec J_{ij}^u \vec S_i^u\vec S_j^u\\
\end{array}
$$

In [663]:
# Computing interaction Hamiltonians for each model
#@title ###### Computing interaction Hamiltonians for each model

Hi = get_H_int(J,S)

if display != "None":
  print("For " + display + " model:")
  print_latex(r"H_{int} = " + as_latex_matrix(np.real(Hi), replace_zeros="0"))

For Heisenberg XXX model:


<IPython.core.display.Math object>

### Total Hamiltonian

$$
H_{T} = H_0 + H_{int}
$$

In [664]:
# Computing total Hamiltonians for each model
#@title ###### Computing total Hamiltonians for each model

H = H0 + Hi

if display != "None":
  print("For " + display + " model:")
  print_latex(r"H_T = " + as_latex_matrix(np.real(H), replace_zeros="0"))

For Heisenberg XXX model:


<IPython.core.display.Math object>

## 2.2 - Diagonalization algorithm

### Ground state

Pseudo code:

$$
\begin{aligned}
&\text { take random vector }\left|\phi_0\right\rangle \\
&\text { normalize }\left|\phi_0\right\rangle \\
&H \leftarrow H-\text { shift } * \operatorname{id}_{2^N} \\
&\text { while } \| H\left|\phi_0\right\rangle-\left\langle\phi_0|H| \phi_0\right\rangle\left|\phi_0\right\rangle \|>\epsilon \text { and } k \leq k_{\max } \text { do } \\
&\quad\left|\phi_0\right\rangle \leftarrow H\left|\phi_0\right\rangle \\
&\quad \text { normalize }\left|\phi_0\right\rangle \\
&\quad k \leftarrow k+1 \\
&\text { end while } \\
&H \leftarrow H+\text { shift } * \operatorname{id}_{2^N} \\
&\lambda_0 \leftarrow\left\langle\phi_0|H| \phi_0\right\rangle
\end{aligned}
$$

The parameters of the algorithm are
- $\text{shift}$ (positive real number): the shifting value used to ensure a negative spectrum;
- $\epsilon$: the wanted precision concerning the verification of the eigenequation (typically $\epsilon$ = 10−8);
- $k_{max}$: the maximal number of iterations, if $k$ reaches $k_{max}$ before the eigenequation be satisfied with
the precision $\epsilon$ then the algorithm has not converged.

In [665]:
# Ground state

def average(operator, state):
    return state.conj().T @ operator @ state

def ground_state(H, shift = None, kmax=10000, eps=1e-8):
    if shift is None:
        shift = int(abs(np.amax(H)) * 10)
        
    phi = np.exp(1j * np.random.rand(2**N) * 2 * np.pi)
    phi = phi / np.linalg.norm(phi)
    H = H - shift * np.eye(2**N)
    k = 0
    while np.linalg.norm(H @ phi - average(H, phi)*phi) > eps and (k <= kmax):
        phi = H @ phi
        phi = phi / np.linalg.norm(phi)
        k += 1
    H = H + shift * np.eye(2**N)
    print(f"Converged in {k} iterations")
    return phi, average(H, phi)

phi0, l0 = ground_state(H)

if display != "None":
    print("For " + display + " model:")
    print_latex(r"\phi_0 = "
        + as_latex_vector(phi0, replace_zeros="0")
        + r"\quad \lambda_0 = "
        + f"{np.real(l0):.3f}"
        + r"\quad \text{and using numpy:} \quad \phi_0 = "
        + as_latex_vector(np.linalg.eigh(H)[1][0], replace_zeros="0")
        + r"\quad \lambda_0 = "
        + f"{np.linalg.eigh(H)[0][0]:.3f}")

Converged in 63 iterations
For Heisenberg XXX model:


<IPython.core.display.Math object>

### First excited state

Pseudo code:
$$
\begin{aligned}
&\text { take random vector }\left|\phi_1\right\rangle \\
&\left|\phi_1\right\rangle \leftarrow\left|\phi_1\right\rangle-\left\langle\phi_0 \mid \phi_1\right\rangle\left|\phi_0\right\rangle \\
&\text { normalize }\left|\phi_1\right\rangle \\
&H \leftarrow H-\text { shift }^* \text { id }_{2^N} \\
&\text { while } \| H\left|\phi_1\right\rangle-\left\langle\phi_1|H| \phi_1\right\rangle\left|\phi_1\right\rangle \|>\epsilon \text { and } k<k_{\text {max }} \text { do } \\
&\quad \phi_1 \leftarrow H \phi_1 \\
&\quad\left|\phi_1\right\rangle \leftarrow\left|\phi_1\right\rangle-\left\langle\phi_0 \mid \phi_1\right\rangle\left|\phi_0\right\rangle \\
&\quad \text { normalize }\left|\phi_1\right\rangle \\
&\quad k \leftarrow k+1 \\
&\text { end while } \\
&H \leftarrow H+\text { shift } * \text { id }_{2^N} \\
&\lambda_1 \leftarrow\left\langle\phi_1|H| \phi_1\right\rangle
\end{aligned}
$$

In [666]:
# First excited state

def first_excited_state(H, phi0, shift = None, kmax=10000, eps=1e-8):
    if shift is None:
        shift = 100 #int(abs(np.amax(H)) * 10)
    
    phi = np.exp(1j * np.random.rand(2**N) * 2 * np.pi)
    phi = phi - phi0.conj().T @ phi * phi0
    phi = phi / np.linalg.norm(phi)

    H = H - shift * np.eye(2**N)

    k = 0
    while np.linalg.norm(H @ phi - average(H, phi)*phi) > eps and (k < kmax):
        phi = H @ phi
        phi = phi - phi0.conj().T @ phi * phi0
        phi = phi / np.linalg.norm(phi)
        k += 1
    print(f"Converged in {k} iterations")
    H = H + shift * np.eye(2**N)
    return phi, average(H, phi)

phi1, l0 = first_excited_state(H, phi0)

if display != "None":
    print("For " + display + " model:")
    print_latex(r"\phi_1 = " + as_latex_vector(phi1, replace_zeros="0") + r"\quad \lambda_1 = " + f"{np.real(l0):.3f}"
    + r"\quad \text{and using numpy:} \quad \phi_1 = " + as_latex_vector(np.linalg.eigh(H)[1][1], replace_zeros="0") + r"\quad \lambda_1 = " + f"{np.linalg.eigh(H)[0][1]:.3f}")



Converged in 3961 iterations
For Heisenberg XXX model:


<IPython.core.display.Math object>